In [1]:
import pandas as pd
import numpy as np
import matplotlib.pyplot as plt
import seaborn as sns
import os
import re
import gen_wrangle1 as gw
#import train test split from sklearn
from sklearn.model_selection import train_test_split
#import stat tests
import scipy.stats as stats
from scipy.stats import pearsonr, spearmanr, ttest_ind, chi2_contingency
from sklearn.feature_selection import SelectKBest, f_regression, RFE
from sklearn.linear_model import LinearRegression, LassoLars, TweedieRegressor
from sklearn.preprocessing import PolynomialFeatures
from sklearn.preprocessing import MinMaxScaler, RobustScaler, StandardScaler
from sklearn.metrics import mean_squared_error, r2_score, explained_variance_score
from sklearn.model_selection import GridSearchCV
#import maxabs_scale
from sklearn.preprocessing import MaxAbsScaler
#import
from sklearn.model_selection import cross_val_score
from math import sqrt
import auto_model3 as am
import warnings
warnings.filterwarnings("ignore")

In [2]:
#asks user for a csv file or xlsx name and returns a dataframe
def get_csv_or_excel():
    file_name = input("What is the name of the csv or xlsx file you want to use for the features?")
    if file_name.endswith('.csv'):
        df = pd.read_csv(file_name)
    elif file_name.endswith('.xlsx'):
        df = pd.read_excel(file_name)
    else:
        print("Sorry, I can't read that file.")
    return df


In [3]:
#asks user for a csv file or xlsx name for the predictions and returns a dataframe
def get_csv_or_excel_pred():
    file_name = input("What is the name of the csv or xlsx file you want to use for the prediction?")
    if file_name.endswith('.csv'):
        this_is_it = pd.read_csv(file_name)
    elif file_name.endswith('.xlsx'):
        this_is_it = pd.read_excel(file_name)
    else:
        print("Sorry, I can't read that file.")
    return this_is_it

In [4]:
target_var = input("What is the name of the target variable?")

In [5]:
df = get_csv_or_excel()

In [6]:
target_var

'adjusted_revenue_B'

In [7]:
this_is_it = get_csv_or_excel_pred()

In [8]:
def split_data(df):
    '''
    Take in a DataFrame and perform a train-test split with a 70/30 ratio.
    Return train and test DataFrames.
    '''
    train, test = train_test_split(df, test_size=0.25, random_state=123)
    return train, test

In [9]:
train, test = split_data(df)

In [10]:
#gets a list of the categorical and numerical columns
col_cat, col_num = gw.cat_and_num_lists(train)

The categorical variables are: 
 ['p_election', 'midterm_election'] 

The continuous variables are: 
 ['adjusted_revenue_B', 'year', 'quarter', 'population', 'median_house_income', 'federal_fund_rate', 'unemp_rate', 'misery_index', 'home_ownership_rate', 'government_spending', 'gdp_deflated', 'violent_crime_rate', 'cpi_all_items_avg', 'avg_temperature', 'avg_precipitation', 'palmer_drought_index', 'eci', 'dow', 's_and_p', 'gas_perc_change', 'prime', 'gini', 'Man_new_order', 'hdi', 'construction_res', 'cli', 'auto_loan', 'velocity_of_money', 'loans_and_leases', 'wti', 'soy', 'brent_oil', 'case_shiller_index', 'number_of_disaster', 'consumer_confidence_index', 'business_confidence_index', 'c_e_s_housing', 'c_e_s_health', 'c_e_s_entertainment', 'ease_of_doing_business', 'wars_started'] 



In [11]:
#a function that runs through a list for shapiro test and a
def normality_test_loop(df_train_name,list_of_passed_features=[], list_of_failed_features=[]):
    col_cat, col_num = gw.cat_and_num_lists(df_train_name)
    alpha = .05
    
    for col in col_num:
        stat, p = stats.shapiro(df_train_name[col])
        if p > alpha:
        
            list_of_passed_features.append(col)

        else: list_of_failed_features.append(col)
        
    return list_of_passed_features, list_of_failed_features   

In [12]:
list_of_passed_features, list_of_failed_features=normality_test_loop(train)

The categorical variables are: 
 ['p_election', 'midterm_election'] 

The continuous variables are: 
 ['adjusted_revenue_B', 'year', 'quarter', 'population', 'median_house_income', 'federal_fund_rate', 'unemp_rate', 'misery_index', 'home_ownership_rate', 'government_spending', 'gdp_deflated', 'violent_crime_rate', 'cpi_all_items_avg', 'avg_temperature', 'avg_precipitation', 'palmer_drought_index', 'eci', 'dow', 's_and_p', 'gas_perc_change', 'prime', 'gini', 'Man_new_order', 'hdi', 'construction_res', 'cli', 'auto_loan', 'velocity_of_money', 'loans_and_leases', 'wti', 'soy', 'brent_oil', 'case_shiller_index', 'number_of_disaster', 'consumer_confidence_index', 'business_confidence_index', 'c_e_s_housing', 'c_e_s_health', 'c_e_s_entertainment', 'ease_of_doing_business', 'wars_started'] 



In [13]:
list_of_passed_features

['gdp_deflated',
 'cpi_all_items_avg',
 'avg_precipitation',
 'palmer_drought_index',
 'eci',
 's_and_p',
 'wti',
 'c_e_s_entertainment',
 'wars_started']

In [14]:
list_of_failed_features

['adjusted_revenue_B',
 'year',
 'quarter',
 'population',
 'median_house_income',
 'federal_fund_rate',
 'unemp_rate',
 'misery_index',
 'home_ownership_rate',
 'government_spending',
 'violent_crime_rate',
 'avg_temperature',
 'dow',
 'gas_perc_change',
 'prime',
 'gini',
 'Man_new_order',
 'hdi',
 'construction_res',
 'cli',
 'auto_loan',
 'velocity_of_money',
 'loans_and_leases',
 'soy',
 'brent_oil',
 'case_shiller_index',
 'number_of_disaster',
 'consumer_confidence_index',
 'business_confidence_index',
 'c_e_s_housing',
 'c_e_s_health',
 'ease_of_doing_business']

In [15]:
target_var

'adjusted_revenue_B'

In [16]:
#if it is on the list of passed features run the pearson test
def pearson_test(df_train_name, target_var, list_of_passed_features, pass_stat_list=[], fail_stat_list=[]):
    alpha = 0.05
    for col in list_of_passed_features:
        stat, p = stats.pearsonr(df_train_name[col], df_train_name[target_var])
        if p < alpha:
            pass_stat_list.append(col)
        else:
            fail_stat_list.append(col)
    return pass_stat_list

In [17]:
pass_stat_list=pearson_test(train, target_var, list_of_passed_features)

In [18]:
#if it is on the list of failed features run the spearman test
def spearman_test(df_train_name, target_var, list_of_failed_features, pass_stat_list, fail_stat_list=[]):
    alpha = 0.05
    for col in list_of_failed_features:
        stat, p = stats.spearmanr(df_train_name[col], df_train_name[target_var])
        if p < alpha:
            pass_stat_list.append(col)
        else:
            fail_stat_list.append(col)
    return pass_stat_list

In [19]:
pass_stat_list = spearman_test(train, target_var, list_of_failed_features, pass_stat_list)

#if it is on the categorical list run the test agains the target variable
def ttest(df_train_name, target_var, col_cat, pass_stat_list, fail_stat_list=[]):
    alpha = 0.05
    for col in col_cat:
        target_var = df_train_name[target_var]
        col = df_train_name[col]
        t, p = stats.ttest_ind(target_var, col, equal_var = True)
        if p > alpha:
            pass_stat_list.append(col)
        else:
            fail_stat_list.append(col)
    return pass_stat_list

In [20]:
def ttest_test(df_train_name, target_var, pass_stat_list, fail_stat_list=[]):
    for col in col_cat:
        if col not in pass_stat_list:
            gw.single_pairwise_ind_text(df_train_name, target_var, col, pass_stat_list)
    return pass_stat_list        

In [21]:
#remove the target variable,year, from the list 
pass_stat_list.remove(target_var)
pass_stat_list.remove('year')

In [6]:
#show all columns of this_is_it
pass_stat_list


NameError: name 'pass_stat_list' is not defined

In [23]:
this_is_it = this_is_it[[pass_stat_list]]


KeyError: "None of [Index([('gdp_deflated', 'cpi_all_items_avg', 'eci', 'population', 'median_house_income', 'misery_index', 'government_spending', 'violent_crime_rate', 'prime', 'gini', 'hdi', 'cli', 'velocity_of_money', 'soy', 'consumer_confidence_index', 'c_e_s_health', 'ease_of_doing_business')], dtype='object')] are in the [columns]"

In [ ]:
this_is_it

,gdp_deflated,cpi_all_items_avg,eci,adjusted_revenue_B,year,population,median_house_income,misery_index,government_spending,violent_crime_rate,prime,gini,hdi,cli,velocity_of_money,soy,consumer_confidence_index,c_e_s_health,ease_of_doing_business
0,130.8,301.836,159.5,NaN,2023,334.44,82222.021429,8.48692,6512.501649,384.75,7.82,41.1,0.921,98.902607,1.222,553.159437,97.343637,5751.208898,7


In [ ]:
X_train, X_test = train[[pass_stat_list]], test[[pass_stat_list]]

In [ ]:
X_train.head()

,gdp_deflated,cpi_all_items_avg,eci,adjusted_revenue_B,year,population,median_house_income,misery_index,government_spending,violent_crime_rate,prime,gini,hdi,cli,velocity_of_money,soy,consumer_confidence_index,c_e_s_health,ease_of_doing_business
1,128.269,296.808,155.8,45.232250,2022,333.640000,80793.111557,11.71485,6233.072139,405.0,5.73,39.4,0.921,99.277920,1.165,552.586005,96.364630,5756.706166,7
24,106.487,241.432,127.3,48.994948,2016,323.910000,84237.593248,6.75080,5280.566169,397.9,3.64,41.1,0.922,99.671985,1.448,474.270122,100.890333,5828.401819,7
65,90.628,202.900,102.5,60.931084,2006,298.905385,96990.568396,6.51207,4173.951270,479.3,8.25,41.4,0.900,101.093573,2.017,299.706082,99.864470,4159.346939,3
45,98.595,226.889,114.2,45.859909,2011,311.890000,81260.557594,12.81262,5088.309714,387.1,3.25,40.9,0.911,99.202021,1.714,660.149854,96.981693,4455.157002,5
60,93.327,210.036,106.7,61.856846,2007,302.285879,94080.174105,9.10881,4345.990342,471.8,7.33,40.8,0.906,101.419728,1.977,614.588650,99.074943,4144.413229,3


In [ ]:
y_train, y_test = train[target_var], test[target_var]   

In [ ]:
def scale_data(X_train, X_test, this_is_it):
     #making our scaler
    scaler = MinMaxScaler()
    #fitting our scaler 
    # AND!!!!
    #using the scaler on train
    X_train_scaled = scaler.fit_transform(X_train)
    #using our scaler on test
    X_test_scaled = scaler.transform(X_test)

    # Convert the array to a DataFrame
    X_train_scaled = pd.DataFrame(X_train_scaled, columns=X_train.columns, index=X_train.index)
    
    # Convert the array to a DataFrame
    X_test_scaled = pd.DataFrame(X_test_scaled,columns=X_test.columns, index=X_test.index)

    this_is_it_scaled = scaler.fit_transform(this_is_it)
    this_is_it_scaled = pd.DataFrame(this_is_it_scaled, columns=this_is_it.columns, index=this_is_it.index)  
    
    

    return X_train_scaled, X_test_scaled, this_is_it_scaled

In [ ]:
X_train_scaled, X_test_scaled, this_is_it_scaled = scale_data(X_train, X_test, this_is_it)

In [ ]:
am.auto_regress_grid( y_train, train, X_train_scaled, y_test, X_test_scaled, this_is_it_scaled)

ValueError: Input contains NaN, infinity or a value too large for dtype('float64').